In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scanpy as sc
import seaborn as sns
import os

import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm
from celloracle import motif_analysis as ma
import celloracle as co

/data/jiangjunyao/miniconda3/envs/normal/lib/python3.9/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/data/jiangjunyao/miniconda3/envs/normal/lib/python3.9/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/d

In [10]:
#plt.rcParams["font.family"] = "arial"
plt.rcParams["figure.figsize"] = [6,6]
%config InlineBackend.figure_format = 'retina'
plt.rcParams["savefig.dpi"] = 600

%matplotlib inline

In [ ]:
peak = pd.read_csv('/data/jiangjunyao/polyATAC/nmp_final/ct_p2g/all_p2g.csv',index_col=0)
peak.columns = ['peak_id','gene_short_name']
tfi = ma.TFinfo(peak_data_frame=peak,
            ref_genome='mm10',
            genomes_dir=None)
tfi.scan(fpr=0.02,
     motifs=None,  # If you enter None, default motifs will be loaded.
     verbose=True)
tfi.reset_filtering()
# Do filtering
tfi.filter_motifs_by_score(threshold=10)
# Format post-filtering results.
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)
df = tfi.to_dataframe()

In [50]:
df.to_csv('/data/jiangjunyao/polyATAC/nmp_final/ct_p2g/ct_p2g_baseGRN.csv')

In [3]:
### feature selection
adata1 = sc.read_h5ad('/data/jiangjunyao/polyATAC/nmp_final/nmp_sp_pm_all_sample.h5ad')
nmp_deg = pd.read_csv('/data/jiangjunyao/polyATAC/nmp_final/nmp_pseudotime_module/only_rna/group_df.csv',index_col=0)
sc.pp.normalize_total(adata1)
sc.pp.log1p(adata1)
sc.pp.highly_variable_genes(adata1,n_top_genes=2000,subset=False)
nmp_deg = nmp_deg.index.tolist()
nmp_hvg = adata1.var_names[adata1.var['highly_variable']].tolist()
all_gene = nmp_deg+nmp_hvg
all_gene = list(set(all_gene))
adata_use = adata1[:,all_gene]
print(adata_use)
base_GRN = pd.read_csv('/data/jiangjunyao/polyATAC/nmp_final/ct_p2g/ct_p2g_baseGRN.csv')
def run_celloracle(adata,baseGRN=None,group='celltype',
                       embedding_name='X_umap',ncores=10,filter=None):
        print('build grn based on celloracle')
        oracle = co.Oracle()
        oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name=group,
                                   embedding_name=embedding_name)
        oracle.import_TF_data(TF_info_matrix=baseGRN)
        oracle.pcs=adata_use.obsm['X_scvi']
        #n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
        n_cell = oracle.adata.shape[0]
        k = 50
        print(f"Auto-selected k is :{k}")
        oracle.knn_imputation(n_pca_dims=10, k=k, balanced=True, b_sight=k*8,
                              b_maxl=k*4, n_jobs=ncores)
        return oracle
sc.pp.normalize_total(adata_use)
oracle=run_celloracle(adata_use,base_GRN,'fatebias')
links = oracle.get_links(cluster_name_for_GRN_unit='fatebias', alpha=10,
                         verbose_level=10)

View of AnnData object with n_obs × n_vars = 29633 × 2077
    obs: 'nCount_RNA', 'nFeature_RNA', 'Batch', 'celltype', 'barcodes', 'pgen', 'rare_barcodes', 'fatebias'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'X_scvi', 'X_umap'
build grn based on celloracle
Auto-selected k is :50


  0%|          | 0/5 [00:00<?, ?it/s]

Inferring GRN for Paraxial mesoderm...


  0%|          | 0/792 [00:00<?, ?it/s]

Inferring GRN for Paraxial_mesoderm_bias...


...

  0%|          | 0/792 [00:00<?, ?it/s]

Inferring GRN for Spinal_cord_bias...


  0%|          | 0/792 [00:00<?, ?it/s]

Inferring GRN for other...


  0%|          | 0/792 [00:00<?, ?it/s]

In [4]:
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)
links.get_network_score()

In [49]:
links.merged_score.to_csv('/data/jiangjunyao/polyATAC/nmp_final/celloracle_grn/grn_score.csv')